# DATA_SpecimenManagement.ipynb
Created by: `Panawun P.` <br>
Created on: `2025-10-22`<br>
Last editted: `2025-10-26`<br>
<br>
Manage specimen data in raw data file, create specimen ID from identification data, auto manage specimen photo album and add link to data file.
- Data has to be sorted by Date > Time before running the script

---

## Setting and Init

In [ ]:
RAW_DATA_SHEET_ID = '[GOOGLE DRIVE FILE ID]'

SPECIMEN_SHEET_NAME = 'SpecimenData'
SPECIMEN_IMG_FOLDER_ID = '[GOOGLE DRIVE FOLDER ID]'

In [ ]:
from datetime import datetime, timezone
from zoneinfo import ZoneInfo
import pytz
import time

# OAuth for gspread
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()

import requests
import gspread

from googleapiclient.errors import HttpError
from googleapiclient.discovery import build
drive_service = build('drive', 'v3', credentials=creds)

gc = gspread.authorize(creds) # Authorize with OAuth

In [ ]:
# ect settings
RUNNING_NO_LEN = 3 # Numbers of digits for running numbers
IMG_NO_LEN = 2

## Functions

### fn: open_datasheet
Open spreadsheet and return spreadsheet ID, worksheet ID, and data as variable

In [ ]:
def open_datasheet():
    # Read data
    sheet_name = SPECIMEN_SHEET_NAME
    sheet_url = f'https://docs.google.com/spreadsheets/d/{RAW_DATA_SHEET_ID}/'

    spreadsheet = gc.open_by_url(sheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_values()

    return spreadsheet, worksheet, data

### fn: generate_specimen_id

In [ ]:
def generate_specimen_id(row_info, row, taxa_count):
    # If row has all info to generate ID
    if row[row_info['transect_id']] and row[row_info['col_method']] and row[row_info['taxa_fam']]:
        transect_code = row[row_info['transect_id']][-5:].replace('-', '').upper()
        method_code = row[row_info['col_method']][:2].upper()
        family_code = row[row_info['taxa_fam']][:2].upper()

        # Running number for family
        if family_code not in taxa_count:
            taxa_count[family_code] = 1
        else:
            taxa_count[family_code] += 1
        running_no = f'{taxa_count[family_code]:0{RUNNING_NO_LEN}d}'

        row[row_info['specimen_id']] = f'{transect_code}{method_code}-{family_code}{running_no}'
        # print(specimen_id)
        return row

    else: # Skip rows with incomplete info
        return False

### fn: create_specimen_img_folder

In [ ]:
def create_specimen_img_folder(specimen_id):
    folder_metadata = {'name': specimen_id,
                       'mimeType': 'application/vnd.google-apps.folder',
                       'parents': [SPECIMEN_IMG_FOLDER_ID]}

    # Check if folder already exist, return folder id
    query = f"name='{folder_metadata['name']}' and mimeType='{folder_metadata['mimeType']}' and '{SPECIMEN_IMG_FOLDER_ID}' in parents and trashed =false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    items = results.get('files', [])
    if items: return items[0]['id']

    # If folder does not exist, attempt to create folder
    try:
        print("Creating image folder...")
        folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
        return folder['id']
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

### fn: add_info_to_sheet

In [ ]:
def add_info_to_sheet(worksheet, row_info, row_i, specimen_id, folder_id):
    # Convert number to letter
    int_to_letter = lambda x:"" if x==0 else int_to_letter((x-1)//26)+chr((x-1)%26+ord("A"))

    # Update sheet with specimen id
    cell_col = cell_col = row_info['specimen_id'] # Determine cell location
    cell_location = f'{int_to_letter(cell_col+1)}{row_i+1}'
    worksheet.update(cell_location, [[specimen_id]], value_input_option='USER_ENTERED')

    # Update sheet with folder link
    cell_col = cell_col = row_info['specimen_img'] # Determine cell location
    cell_location = f'{int_to_letter(cell_col+1)}{row_i+1}'
    drive_folder_link = f'https://drive.google.com/drive/folders/{folder_id}'
    link_formula = f'=HYPERLINK("{drive_folder_link}", "{specimen_id}")'
    worksheet.update(cell_location, [[link_formula]], value_input_option='USER_ENTERED')

### fn: parse_img_time

In [ ]:
def parse_img_time(img_list):
    converted_img_list = []
    for img_i, img_info in enumerate(img_list):
        converted_img_list.append(img_info)
        # Parse date and time from the image assume Pixel photo file format
        # eg. "PXL_20251020_234817441.jpg" - Note that time in file name is always in UTM
        datetime_string = img_info['name'][4:-9]
        datetime_format = '%Y%m%d_%H%M'
        img_info['datetime'] = datetime.strptime(datetime_string, datetime_format)
        img_info['datetime'] = img_info['datetime'].replace(tzinfo=timezone.utc) # Make datetime obj UTC-aware

        # Convert from UTC to Canada/Eastern
        converted_img_list[img_i]['datetime'] = img_info['datetime'].astimezone(ZoneInfo('Canada/Eastern'))

    # print(converted_img_list[0]['datetime'])

    return converted_img_list

### fn: get_img_list

In [ ]:
def get_img_list(folder_id):
    query = f"'{folder_id}' in parents and trashed =false"
    file_list = drive_service.files().list(q=query,
                                        fields='nextPageToken, files(id, name, mimeType)').execute()
    img_list = file_list.get('files', [])

    img_list = [file_info for file_info in img_list if 'image/' in file_info['mimeType']]
    img_list = parse_img_time(img_list)

    return img_list

## Main Block

### fn: update_sheet

In [ ]:
def update_sheet():
    taxa_count = {} # To keep running numbers/count for specimen id
    unsort_img_list = get_img_list(SPECIMEN_IMG_FOLDER_ID) # get a list of unsorted imgs
    print(f"Total {len(unsort_img_list)} images to upload.")

    for row_i, row in enumerate(data):
        if row_i == 0: # Header row
            row_info = {'id_date': row.index('ID DATE'),
                        'id_time': row.index('ID TIME'),
                        'specimen_id': row.index('SPECIMEN ID'),
                        'specimen_img': row.index('SPECIMEN IMAGES'),
                        'col_method': row.index('METHOD'),
                        'transect_id': row.index('TRANSECT ID'),
                        'taxa_fam': row.index('FAMILY'),
                        }
        else:
            # Generate specimen id, create img folder, and add img folder link to sheet
            row = generate_specimen_id(row_info, row, taxa_count)

            if row: # Check that specimen ID is generated (data not missing)
                print(f"Processing specimen: {row[row_info['specimen_id']]}")
                folder_id = create_specimen_img_folder(row[row_info['specimen_id']])

                # Add link to img folder to sheet
                add_info_to_sheet(worksheet, row_info, row_i, row[row_info['specimen_id']], folder_id)
            else:
                continue # Skip further processing on the row if data missing

            # Go through images, uploaded to the folder, sort into folder, and rename files
            datetime_format = '%Y-%m-%d %H:%M'
            id_datetime = f"{row[row_info['id_date']]} {row[row_info['id_time']]}"
            next_id_datetime = f"{data[row_i+1][row_info['id_date']]} {data[row_i+1][row_info['id_time']]}"

            # Convert to datetime obj
            id_datetime = datetime.strptime(id_datetime, datetime_format)
            if next_id_datetime == " ": # If next row is empty then use current datetime
                next_id_datetime = datetime.now()
            else:
                next_id_datetime = datetime.strptime(next_id_datetime, datetime_format)
            # print(f"id_datetime: {id_datetime}, next_id_datetime: {next_id_datetime}")

            # Reformat to timezone-aware datetime obj
            id_datetime = pytz.timezone('Canada/Eastern').localize(id_datetime)
            next_id_datetime = pytz.timezone('Canada/Eastern').localize(next_id_datetime)
            print(f"ID Time: {id_datetime} to {next_id_datetime}")

            # Get list of images assoc with specimen ID time
            specimen_img_list = [img_info for img_info in unsort_img_list if id_datetime <= img_info['datetime'] < next_id_datetime]

            if specimen_img_list:
                print(f"Uploading {len(specimen_img_list)} specimen images...")
                # Rename images and move to specimen image folder
                for img_i, img_info in enumerate(specimen_img_list):
                    img_running_no = f'{img_i+1:0{IMG_NO_LEN}d}'
                    name_metadata = {'name': f"{row[row_info['specimen_id']]}_{img_running_no}"}

                    updated_file = drive_service.files().update(fileId=img_info['id'],
                                                                body=name_metadata,
                                                                addParents=folder_id,
                                                                removeParents=SPECIMEN_IMG_FOLDER_ID,
                                                                fields='id, name'
                                                                ).execute()

                    time.sleep(1)

In [ ]:
spreadsheet, worksheet, data = open_datasheet()

update_sheet()

Total 35 images to upload.
Processing specimen: R02DR-TH001


/tmp/ipython-input-3513627340.py:8: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(cell_location, [[specimen_id]], value_input_option='USER_ENTERED')
/tmp/ipython-input-3513627340.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(cell_location, [[link_formula]], value_input_option='USER_ENTERED')


ID Time: 2025-10-07 23:15:00-04:00 to 2025-10-07 23:58:00-04:00
Processing specimen: R02DR-CL001
ID Time: 2025-10-07 23:58:00-04:00 to 2025-10-09 19:44:00-04:00
Processing specimen: R02DR-SA001
ID Time: 2025-10-09 19:44:00-04:00 to 2025-10-09 21:02:00-04:00
Processing specimen: R02DR-TE001
ID Time: 2025-10-09 21:02:00-04:00 to 2025-10-09 21:50:00-04:00
Processing specimen: R02DR-TE002
ID Time: 2025-10-09 21:50:00-04:00 to 2025-10-09 23:24:00-04:00
Processing specimen: M02DR-TH002
ID Time: 2025-10-09 23:24:00-04:00 to 2025-10-09 23:57:00-04:00
Processing specimen: M02DR-TE003
ID Time: 2025-10-09 23:57:00-04:00 to 2025-10-10 00:09:00-04:00
Processing specimen: M02DR-TE004
ID Time: 2025-10-10 00:09:00-04:00 to 2025-10-20 10:15:00-04:00
Processing specimen: R03DR-SA002
ID Time: 2025-10-20 10:15:00-04:00 to 2025-10-20 10:20:00-04:00
Processing specimen: R03DR-TE005
ID Time: 2025-10-20 10:20:00-04:00 to 2025-10-20 10:25:00-04:00
Processing specimen: R03DR-DI001
ID Time: 2025-10-20 10:25:00-0